In [ ]:
# B2C 단가 설정 전체 코드 (불필요 컬럼 제거 + 마진율 포함)

import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# 1️⃣ 엑셀 파일 경로 설정
file_path = '/mnt/data/B2C단가설정_최종.xlsx'

# 2️⃣ 데이터 로드 및 숫자 변환

# 분석 대상 컬럼
df = pd.read_excel(file_path)
numeric_cols = ['입고단가', '소비자가', '모모단가', '크로스단가', '솔과장님', '이이사님']

# 숫자 변환 (에러 시 0)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 3️⃣ 불필요 컬럼 제거
제외컬럼 = ['전무님', '이이사님 배송']
for col in 제외컬럼:
    if col in df.columns:
        df = df.drop(columns=[col])

# 4️⃣ 결과 컬럼 추가
df['최종단가'] = 0
df['비고'] = ""
df['적용마진율'] = 0  # % 기준으로 저장

오차범위 = 0.2  # 소비자가 기준 오차범위 20%

# 5️⃣ 단가 산출 로직
for idx, row in df.iterrows():
    기준단가 = row['입고단가']
    마진율 = 0.7  # 70%부터 시작
    최소마진 = 0.3  # 30% 마진까지 허용

    추천단가_우선순위 = [row['이이사님'], row['솔과장님']]
    모모단 = row['모모단가']
    크로스단 = row['크로스단가']
    소비자가 = row['소비자가']

    최종단가 = 0
    최종마진 = 0
    소비자가_상한 = 소비자가 * (1 + 오차범위) if 소비자가 > 0 else np.inf

    # 마진율 반복 감소하며 조건 체크
    while 마진율 >= 최소마진:
        계산단가 = 기준단가 * (1 + 마진율)

        # 소비자가 오차범위 상한 초과 시 제외
        if 계산단가 > 소비자가_상한:
            마진율 -= 0.05
            continue

        # 경쟁사 단가 기준: 둘 다보다 낮아야 통과
        if (모모단 > 0 and 계산단가 >= 모모단) or (크로스단 > 0 and 계산단가 >= 크로스단):
            마진율 -= 0.05
            continue

        # 추천단가 기준 우선순위 체크
        추천검증 = False
        for 추천 in 추천단가_우선순위:
            if 추천 > 0 and (추천 * (1 - 오차범위) <= 계산단가 <= 추천 * (1 + 오차범위)):
                추천검증 = True
                break

        # 추천 조건 통과 시 확정
        if 추천검증 or all(r == 0 for r in 추천단가_우선순위):
            최종단가 = 계산단가
            최종마진 = 마진율
            break

        마진율 -= 0.05

    # 추천·경쟁사 모두 0일 때 소비자가 오차범위 기준으로 최고값 선택
    if 최종단가 == 0 and 모모단 == 0 and 크로스단 == 0 and all(r == 0 for r in 추천단가_우선순위):
        마진시도 = np.arange(0.7, 0.29, -0.05)
        후보단가 = [(m, 기준단가 * (1 + m)) for m in 마진시도
                   if 소비자가 * (1 - 오차범위) <= 기준단가 * (1 + m) <= 소비자가_상한]
        if 후보단가:
            최종마진, 최종단가 = max(후보단가, key=lambda x: x[1])

    df.loc[idx, '최종단가'] = round(최종단가, 2)
    df.loc[idx, '적용마진율'] = round(최종마진 * 100, 1)

    # 경쟁사 단가 초과 시 주의 표시
    if (모모단 > 0 and 최종단가 >= 모모단) or (크로스단 > 0 and 최종단가 >= 크로스단):
        df.loc[idx, '비고'] = "주의"

# 6️⃣ 엑셀로 저장
df.to_excel('/mnt/data/B2C단가_불필요제외_마진포함.xlsx', index=False)

# 7️⃣ 색상 강조 포함 저장
wb = load_workbook('/mnt/data/B2C단가_불필요제외_마진포함.xlsx')
ws = wb.active

최종단가_col = None
비고_col = None
for idx, cell in enumerate(ws[1], start=1):
    if cell.value == '최종단가':
        최종단가_col = idx
    if cell.value == '비고':
        비고_col = idx

fill = PatternFill(start_color='FF9999', end_color='FF9999', fill_type='solid')

for row in range(2, ws.max_row + 1):
    if ws.cell(row=row, column=비고_col).value == "주의":
        ws.cell(row=row, column=최종단가_col).fill = fill

wb.save('/mnt/data/B2C단가_불필요제외_마진포함_색상포함.xlsx')
